<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [1]:
#pip install psycopg2

In [2]:
import pandas as pd
import psycopg2
import warnings 

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [5]:
# Даем команду, игнорировать предуперждения.
# Jupiter ругается на предложенный в исходном шаблоне connection=psycopg2.connect() pd.read_sql_query(query_tables, connection),
# и советует применять другие:
#             "UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. 
#              Other DBAPI2 objects are not tested. Please consider using SQLAlchemy. df_table_names = pd.read_sql_query(query_tables, connection)"
warnings.filterwarnings('ignore')

In [6]:
# Выводим перечень таблиц в схеме
query_tables = f''' SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public'
'''
df_table_names = pd.read_sql_query(query_tables, connection)
df_table_names

,table_name
0,test
1,stored_procedure_prac
2,columns
3,contracts
4,t_tab1
5,t_tab2
6,operations
7,table_1
8,films_recent
9,cities


Похоже, что схема содержит куски таблиц используемых для других проектов.

## 3. Предварительный анализ данных

**1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).**

In [7]:
# текст запроса. Запрашиваем общее количество вакансий(строк с предложением работы)  
query_3_1 = f'''SELECT count(id) 
FROM vacancies
''' 

# Передаем результаты запроса в pandas df и выводим количество вакакнсий
cnt_vacancies = pd.read_sql_query(query_3_1, connection).iloc[0,0]# так как нас интересует дискретное значение, а не датафрейм содержащий это значение - извлекаем его( элемент [0,0]) из таблицы 
print(f'Количество вакансий(строк с предложением работы): {cnt_vacancies}') 
#print(f'Количество вакансий, посчитанное при помощи pandas: {df_vacancies.shape[0]}')

# Дополнительно запрашиваем количество уникальных наименований вакансий
cnt_unique_vacancies = pd.read_sql_query('SELECT count(distinct name) FROM vacancies', connection).iloc[0,0]# считаем количество уникальных наименований вакансий. !!! Опять-таки это не требуется. Сделано просто так
print(f'Количество уникальных наименований вакансий: {cnt_unique_vacancies}, что составляет {round(cnt_unique_vacancies/cnt_vacancies*100,2)}% от общего числа вакансий.')

Количество вакансий(строк с предложением работы): 49197
Количество уникальных наименований вакансий: 21223, что составляет 43.14% от общего числа вакансий.


**Дополнительный предварительный анализ, чтобы понять с какими данными мы имеем дело:**


In [8]:
# Создаем датафрейм с таблицей Vacancies
query_vacancies = f''' SELECT * 
FROM vacancies 
'''
df_vacancies = pd.read_sql_query(query_vacancies, connection)

In [9]:
# Выводим информацию о колонках и типах данных таблицы vacancies.  Делаю для наглядности структуры данных 
query_vacancies_info = f''' SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_schema = 'public' AND table_name = 'vacancies'
'''

df_vacancies_info = pd.read_sql_query(query_vacancies_info, connection)
df_vacancies_info

,column_name,data_type
0,id,integer
1,name,text
2,key_skills,text
3,schedule,text
4,experience,text
5,employment,text
6,salary_from,integer
7,salary_to,integer
8,area_id,integer
9,employer_id,integer


In [10]:
#  Получаем краткое описание таблицы вакансии
df_vacancies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49197 entries, 0 to 49196
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49197 non-null  int64  
 1   name         49197 non-null  object 
 2   key_skills   40741 non-null  object 
 3   schedule     49197 non-null  object 
 4   experience   49197 non-null  object 
 5   employment   49197 non-null  object 
 6   salary_from  21485 non-null  float64
 7   salary_to    12690 non-null  float64
 8   area_id      49197 non-null  int64  
 9   employer_id  49197 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 3.8+ MB


In [11]:
# Выводим 5 верхних значений таблицы vacancies  для наглядности данных 

df_vacancies.head(5)

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740


-------------------------------------------------------------------------------------------------------------------------
На глаза случайно попались строки 2 и 3. Две идентичные вакансии на удаленку от одного работодателя в разных регионах. 
Вероятно это одна и та же вакансия, просто продублирована в разных регионах, чтобы попадаться на глаза соискателю.
Стоит посмотреть, насколько это распространенная ситуация. С одной стороны, возможно стоит брать с работодателя допонительные деньги, 
за размещение дубликатов объявлений, с другой стороны, нужно учитывать такого рода вакансии, при рассчете внутренних метрик.(Если у нас 10 дубликатов вакансии и мы нашли работника на одну из них- мы считаем, что эффективность -10%, а фактически 100. Или при рассчете аналитики: мы считаем, что на рынке 10 свободных вакансий и 5 доступных резюме - мы думаем, что это "рынок работника", а на самом деле(если все эти 10 вакансий - всего-лишь одна вакансия) - "рынок работодателя")     

In [12]:
# Cчитаем количество дублирующихся вакансий
query_dubbled_vacancies = f''' 
SELECT employer_id
       ,name
       ,key_skills
       ,schedule
       ,experience	
       ,employment
       ,salary_from
       ,salary_to
       ,count(area_id) cnt_areas	

FROM vacancies 
Group by employer_id,name,key_skills,schedule,experience,employment,salary_from,salary_to
having count(area_id)>1
'''

df_dubbled_vacancies = pd.read_sql_query(query_dubbled_vacancies, connection)
print(f'Количество ситуаций, когда одна и та-же вакансия продублирована более чем в одном регионе:{df_dubbled_vacancies.shape[0]}.\n Общее количество строк таких вакансий {sum(df_dubbled_vacancies['cnt_areas'])}. \n Таким образом, количество вакансий, фактически не являющихся отдельной вакансией: {sum(df_dubbled_vacancies['cnt_areas'])-df_dubbled_vacancies.shape[0]}, что составляет {round(((sum(df_dubbled_vacancies['cnt_areas'])-df_dubbled_vacancies.shape[0])/df_vacancies.shape[0])*100,2)}% от общего числа вакансий ')
print('')
print('Примеры явного массового дублирования обной и той-же вакансии:')
df_dubbled_vacancies[df_dubbled_vacancies['cnt_areas']>20].head(5) # ситуации явного массового дублирования одной и той-же вакансии во всех регионах


Количество ситуаций, когда одна и та-же вакансия продублирована более чем в одном регионе:4261.
 Общее количество строк таких вакансий 14315. 
 Таким образом, количество вакансий, фактически не являющихся отдельной вакансией: 10054, что составляет 20.44% от общего числа вакансий 

Примеры явного массового дублирования обной и той-же вакансии:


,employer_id,name,key_skills,schedule,experience,employment,salary_from,salary_to,cnt_areas
221,1740,Консультант онлайн-поддержки с техническими зн...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,34
223,1740,Консультант службы поддержки в Yandex Cloud,None,Удаленная работа,Нет опыта,Частичная занятость,27000.0,54000.0,22
233,1740,Младший специалист ручного тестирования,None,Удаленная работа,Нет опыта,Частичная занятость,NaN,NaN,169
241,1740,Разработчик (поисковая выдача),None,Удаленная работа,Нет опыта,Полная занятость,NaN,75000.0,54
243,1740,Сотрудник поддержки поиска Яндекса,None,Удаленная работа,Нет опыта,Частичная занятость,13000.0,NaN,99


--------------------------------------------------------------------------------------------------------
 Огромное количество уникальных наименований вакансий - осложняет сопоставление вакансий и аналитику и заслуживает исследования. 

In [13]:
#В связи с тем, что в таблице вакансий - аномально большое количество уникальных наименований вакансий
#и огромный разброс между средним числом вакансий и максимальным числом вакансий, 
# - выводим таблицу vacancies в разрезе наименование вакансии + количество вакансий по убыванию, чтобы посмотреть с чем это связано.
query_vac_per_name = f'''
SELECT  name 
        ,count(id) as id_cnt
FROM vacancies
group by name
order by id_cnt desc
'''                    
df_vac_per_name = pd.read_sql_query(query_vac_per_name, connection)
df_vac_per_name

,name,id_cnt
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
...,...,...
21218,Генеральный менеджер Проекта,1
21219,Ведущий инженер-программист (1С:УПП),1
21220,Главный специалист IT направления,1
21221,Сотрудник Отдела информационных систем,1


In [14]:
# Гипотеза: Большинство вакансий можно сгруппировать по смыслу выполняемых работником операций
# К примеру, "Специалист службы поддержки" (рядовой сотрудник(не руководитель) службы поддерки, которому звонят пользователи с проблеммами. 
# его задача принять обращение пользователя, если для заявленной проблемы есть скрипт пешения - решить проблему, если нет - перенаправить проблему 
# далее техническим специалистам. основной навык - общение с клиентами) 
query_all_support_servise_spec = f'''
SELECT  name as vacancie
        ,count(id) as vacancies_cnt
FROM vacancies
group by name
having (name like '%оддержк%')  and ((name like '%лужб%')or(name like '%ехнб%')) and (name like '%пециалис%')
order by vacancies_cnt desc

''' 

df_all_support_servise_spec = pd.read_sql_query(query_all_support_servise_spec, connection)
print(f'В базе данных {df_all_support_servise_spec.shape[0]} вариантов написания должности "Специалист службы поддержки".\nИз них один раз использовалось {sum(df_all_support_servise_spec[df_all_support_servise_spec['vacancies_cnt']==1]['vacancies_cnt'])}.\nВсего вакансий "Специалист службы поддержки": {sum(df_all_support_servise_spec['vacancies_cnt'])}  ')
df_all_support_servise_spec

В базе данных 79 вариантов написания должности "Специалист службы поддержки".
Из них один раз использовалось 43.
Всего вакансий "Специалист службы поддержки": 953  


,vacancie,vacancies_cnt
0,Специалист службы поддержки в Яндекс Бизнес,168
1,Специалист службы поддержки Рекламной сети Янд...,139
2,Специалист службы поддержки пользователей,126
3,Специалист службы поддержки с техническими зна...,84
4,Специалист службы поддержки с техническими зна...,75
...,...,...
74,Специалист службы поддержки в онлайн-школу (Уд...,1
75,Специалист Службы информационной поддержки,1
76,Специалист отдела технической поддержки (госсл...,1
77,Специалист службы поддержки пользователей QIWI,1


----------------------------------------------------------------------------------------------------------------------------
Описание зарплатных вилок определение средних значений, квартилей, минимумов максимумов и аномалий  

In [15]:
#  Получаем описание предложенных зарплатных вилок от/до 
#(количество не пустых, среднее, среднеквадратичное отклонение, min значение, maх значение, разбивка по квартилям) :
df_vacancies[['salary_from','salary_to']].describe()

,salary_from,salary_to
count,21485.000000,12690.000000
mean,71064.657901,110536.741923
std,56646.897132,89104.795997
min,30.000000,60.000000
25%,35000.000000,50000.000000
50%,50000.000000,80000.000000
75%,90000.000000,150000.000000
max,1000000.000000,1000000.000000


Данные о предлагаемых зарплатах находятся в плачевном состоянии. 56% полей "Зарплата от" и 74% полей "Зарплата до" - не заполнены. Разброс значений от 30 рублей до 1 000 000. Очень высокое среднеквадратичное отклонение по сравнению с матожиданием.... 

In [16]:
# Примеры выбросов вниз по зарплатным вилкам приполной занятости.
# Предпологаю наличие двух возможных ситуаций для полной занятости:
#                                            1) указано 35, а имеется в виду 35 000. (это объясняет  странные зарплатные вилки  от "42 до 50000", или вакансии "Computer Vision Engineer" с опытом 3-6 лет в Москве с зарплатной вилкой 250-350 рублей)
#                                            2) возможно часть вакансий указана в валюте без указания валюты.
# Помимо этого есть целая палитра возможных вариантов для неполной занятости или проектных заданий
print(f'Общее количество ваканисий с предложением зарплаты ниже 1000: {df_vacancies[((df_vacancies['salary_from']<1000) | (df_vacancies['salary_to']<1000))].shape[0]}')
print('\nПримеры явных выбросов зарплат вниз при полной занятости')
df_vacancies[((df_vacancies['salary_from']<1000) | (df_vacancies['salary_to']<1000))&(df_vacancies['employment']=='Полная занятость')].head(10) 

Общее количество ваканисий с предложением зарплаты ниже 1000: 16

Примеры явных выбросов зарплат вниз при полной занятости


,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
2818,66177314,Инженер по информационной безопасности,Информационная безопасность\tВнедрение систем ...,Полный день,Нет опыта,Полная занятость,35.0,NaN,26,643860
3210,51951335,Системный администратор,None,Полный день,От 3 до 6 лет,Полная занятость,70.0,NaN,78,43906
9794,55385660,Программист,Sass\tjQuery\tAjax\tJavaScript\tHTML,Полный день,От 1 года до 3 лет,Полная занятость,NaN,75.0,23,1729613
18621,53203579,Аналитик 1С,None,Полный день,От 1 года до 3 лет,Полная занятость,80.0,120000.0,99,360319
19524,55069564,Контент-менеджер,Adobe Photoshop\tНаполнение контентом\tГрамотн...,Удаленная работа,От 1 года до 3 лет,Полная занятость,900.0,NaN,1007,2848813
24337,54909932,Middle+/Senior C# разработчик,Git\tPostgreSQL\tООП\tAWS\tAWS Lambda\tElastic...,Полный день,От 3 до 6 лет,Полная занятость,65.0,NaN,159,4823409
24561,55884372,Инженер по обслуживанию ресторанов Макдоналдс ...,Настройка ПО\tНастройка ПК\tРемонт ПК\tТехниче...,Гибкий график,От 1 года до 3 лет,Полная занятость,42.0,50000.0,26,84969
26186,66145376,Помощник IT-специалиста,Adobe Photoshop\tHTML\tWeb\tОпрятный внешний в...,Полный день,Нет опыта,Полная занятость,30.0,NaN,237,3046483
27198,51504693,Системный администратор,TCP/IP\tАдминистрирование сетевого оборудовани...,Полный день,Нет опыта,Полная занятость,80.0,150000.0,2077,44936
28662,55518035,Computer Vision Engineer,Tensorflow\tOpenCV\tMachine Learning\tPython\t...,Полный день,От 3 до 6 лет,Полная занятость,250.0,350.0,1,4484124


In [17]:
# Примеры выбросов вверх по зарплатным вилкам приполной занятости.
# Предпологаю наличие двух возможных ситуаций для полной занятости:
#                                            1) указано 35, а имеется в виду 35 000. (это объясняет  странные зарплатные вилки  от "42 до 50000", или вакансии "Computer Vision Engineer" с опытом 3-6 лет в Москве с зарплатной вилкой 250-350 рублей)
#                                            2) возможно часть вакансий указана в валюте без указания валюты.
# Помимо этого есть целая палитра возможных вариантов для неполной занятости или проектных заданий
print(f'Общее количество ваканисий с предложением зарплаты выше 500 000 при опыте отгода до трех: {df_vacancies[((df_vacancies['salary_from']>500000) | (df_vacancies['salary_to']>500000))&(df_vacancies['experience']=='От 1 года до 3 лет')].shape[0]}')
print('\nПримеры явных выбросов зарплат ввверх')
df_vacancies[((df_vacancies['salary_from']>500000) | (df_vacancies['salary_to']>500000))&(df_vacancies['experience']=='От 1 года до 3 лет')].head(10)

Общее количество ваканисий с предложением зарплаты выше 500 000 при опыте отгода до трех: 4

Примеры явных выбросов зарплат ввверх


,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
3499,66097724,QA Automation Engineer,Тестирование\tHTML\tJavaScript\tGit\tТестирова...,Полный день,От 1 года до 3 лет,Полная занятость,400000.0,1000000.0,160,703174
29187,55052597,"Программист-разработчик C#, SQL",C#\tASP.NET\tWPF\t.NET Framework\tJavaScript\t...,Полный день,От 1 года до 3 лет,Полная занятость,40000.0,600000.0,88,1830260
29188,55052598,"Программист-разработчик C#, SQL",C#\tASP.NET\tWPF\t.NET Framework\tJavaScript\t...,Полный день,От 1 года до 3 лет,Полная занятость,40000.0,600000.0,90,1830260
33023,53692636,Брокер по продаже апартаментов,Ведение переговоров\tТелефонные переговоры\tРа...,Полный день,От 1 года до 3 лет,Полная занятость,500000.0,1000000.0,237,1625845


**Краткие выводы по предварительному анализу таблицы 'vacancies'**:
1) Количество вакансий(строк с предложением работы): 49197
2) Количество уникальных наименований вакансий - 21223, что составляет 43.14% от общего числа вакансий. Это очень много! 
Мы все, конечно уникальные снежинки, но построить бизнес процесс hr-агентства без сопоставления требований работодателя с имеющимися резюме, а так-же с вакансиями конкурентов на рынке - фактически невозможно. Условный "специалист службы поддержки"- работа имеющая примерно одинаковый базовый фукционал вне зависимости от того, каким именно образом эта должность записана в резюме и ли в карточке вакансии. Мое пердположение состоит в том, что таблица сопоставления 'названия вакансий' и 'сути вакансии' у hh.ru существует, просто она находится за пределами предоставленных нам данных.
3) Примерно 20.4% выкансий - вероятно не является вакансиями, а является дубликатами вакансий (от одного и того-же работодателя с идентичными условиями, просто в разных регионах). 
4) У 17% вакансий (8456 из 49197) не заполнено поле ключевые навыки. Это одновременно сильно ухудшает качество поиска претендента для клиета, и сильно ухудшает качество входных данных для аналитики hh. Бизнес hh состоит из двух составляющих - поиск кандидатов на работу и продажа аналитики по рынку труда. Наличие незаполненных полей - вредит обоим направлениям. Наверно, стоит подтаталкивать работодателя заполнять как можно больше полей, возможно сделать часть полей обязательными к заполнению.
5) Поля отвечающие за зарплатную вилку - дискретные заначения. Разумно было бы иметь структуру данных, реализующую возможность вносить изменения в зарплату без потерь ранее указанных данных. Условно с 01.01.2024 предприятие искало бухгалтера с зарплатной вилкой от 50 до 70 т.р., а потом, не найдя никого, - повысило поменяло предложение по зарплате на от 70 до 100 т.р. Ситуация в жизни возможная. Хорошо бы если бы имеющаяся система хранения данных позволяла ее учитывать.
6) 56% полей "Зарплата от" и 74% полей "Зарплата до" - не заполнены. Помимо этого есть явные выбросы по зарплатам. Есть зарплаты заведомо ниже реалистичных (возможные варианты: зарплата подразумевается в тысячах рублей, но это не указано/ зарплата подразумевается в валюте, но это не указано/ зарплата указана за нестандартизированфый объем работы (не за месяц, а за день/час/неделю/штуку/проект и т.д.)) 
7) В структуре данных отсутсвуют поля отвечающие за дату открытия вакансии, дату закрытия вакансии, причины закрытия вакансии(условно нашли сотрудника/ отозвали объявление/истек срок ожидания и т.д.). Без этих данных в принципе не возможно организовать базу данных рекрутингового агнтсва на любом отрезке времени, кроме дискретного времени.
   
**Гипотеза**

Моя гипотеза состоит в том, что перед нами не база данных HH.RU.

Это либо не очень удачная имитация данных рекрутингового агентсва,
либо это некий слайс данных, которые HH.RU предоставляет в открытый доступ(или для учебнах целей), но с обрезанной частью структуры данных, позволяющией эффективно организовывать работу hr-агентства и получать разумную аналитику.

Лично я склоняюсь ко второму варианту.

Если же предположить, что имеющаяся база данных - это вся имеющаяся база некого условного hr-агентства - я бы порекомендовал следующее:
* привести вакансии к виду, который позволит их сопоставлять (либо внедрить  рекомендательную систему (пока работодатель печатает название вакансии предлагать ему уже ренее имевшиеся варианты названия), либо внутренний алгоритм определения соответствия 'названия вакансии' и 'сути вакансии')
* настаивать на заполнении полей с ключевыми навыками
* настоятельно рекомендовать работадателям заполнять зарплатные вилки + на стадии ввода данных проверять, не является ли сумма подозрительно маленькой, или подозрительно большой и сообщать об этом работодателю.
* ввести контроль дубликатов вакансий.
* обязательно завести поля, позволяющие понимать, является ли вакасия открытой, или закрытой. хранить данные о том, когда вакансия была открыта, когда пролонгированиа, когда закрыта, и почему.
* учитывая насколько неинформативна имеющаяся база данных - обратиться к бизнес консультанту и собрать отраслевые best practices, а так-же подумать о целесообразности системного инженера. 


-----------------

**2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).**

In [18]:
# текст запроса
query_3_2 = f'''SELECT count(id) 
FROM employers
'''
# результат запроса
cnt_employers = pd.read_sql_query(query_3_2, connection).iloc[0,0]
cnt_employers_from_vac = pd.read_sql_query('SELECT count(distinct employer_id) FROM vacancies', connection).iloc[0,0]
print(f'Количество работодателей: {cnt_employers}') 

print(f'Количество работодателей имеющих открытые вакансии: {cnt_employers_from_vac}, что составляет {round((cnt_employers_from_vac/cnt_employers)*100,2)} % от общего числа работодателей в справочнике.')

Количество работодателей: 23501
Количество работодателей имеющих открытые вакансии: 14906, что составляет 63.43 % от общего числа работодателей в справочнике.


In [19]:
# передаем всю таблицу  employers в pandas
query_employers = f''' SELECT * 
FROM employers 
'''

df_employers = pd.read_sql_query(query_employers, connection)


In [20]:
#  Получаем краткое описание таблицы employers
df_employers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23501 entries, 0 to 23500
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      23501 non-null  int64 
 1   name    23501 non-null  object
 2   area    23501 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 550.9+ KB


In [21]:
# Выводим 5 верхних значений таблицы employers для наглядности данных 
df_employers.head(5)

,id,name,area
0,2393,"Программный Продукт, ИТ-компания",1
1,72977,БАРС Груп,88
2,3155,"Бест, Торгово-производственная компания, Екате...",3
3,675,КОРУС Консалтинг,2
4,1840010,филиал ФКУ Налог-Сервис ФНС России в Республик...,88


**Краткий вывод по справочнику 'employers'**

Обычный справочник без какой-то группировки. Просто id, наименование, и регион(предположительно регион регистрации). 

Нет идентификаторов юр лиц (ИНН или ОГРН(ОГРНИП)). 

Нет привязки к контактной информации (условно дополнительного справочника контактов, кто руководитель, с кем общались, кому счета высталять и т.д.). 

Думаю, что все это в достатке есть у HH.ru, просто мы работаем с усеченным куском данных.

**Из важного:**
- Только 63.43 % работодателей в справочнике 'employers' задейсвовано в таблице вакансий( это косвенно подтверждает предположение, что мы работаем не с полными данными, а со слайсом данных от HH.ru )
- Справочник заполнен полностью, пропусков и пустых занчений нет.
- Важно отметить: Справочник 'employers', равно как и таблица 'vacancies' имеет поле 'area'(в обоих случаях идет ссылка на один и тот же справочник), но эти поля имеют разные смысловые значение, которые не стоит путать. В таблице 'vacancies'  поле 'area' обозначает регион в котором гнанимаются работники, а в справочнике 'employers' поле 'area' обозначает регион в котором предприятие зарегистрировано. Эти поля могут не иметь к друг дргу никакого отношения.

-------------------------------

**3. Посчитайте с помощью запроса количество регионов (таблица `areas`).**

In [22]:
# текст запроса
query_3_3 = f'''SELECT count(id) 
FROM areas
'''
# результат запроса
cnt_areas = pd.read_sql_query(query_3_3, connection).iloc[0,0]

print(f'Количество регионов, посчитанное SQL запросом: {cnt_areas}') 


Количество регионов, посчитанное SQL запросом: 1362


In [23]:
#Выводим 5 верхних значений таблицы для наглядности.
query_areas = f''' SELECT * 
FROM areas 

'''

df_areas = pd.read_sql_query(query_areas, connection)
df_areas.head(5)

,id,name
0,2758,Тбилиси
1,8,Майкоп
2,1180,Нерюнгри
3,1240,Новокузнецк
4,2,Санкт-Петербург


In [24]:
#  Получаем краткое описание таблицы areas
df_areas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1362 entries, 0 to 1361
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1362 non-null   int64 
 1   name    1362 non-null   object
dtypes: int64(1), object(1)
memory usage: 21.4+ KB


In [25]:
# Запрашиваем только те  регионы, которые встречаются в вакансиях
query_areas_in_vacancies  = f'''
SELECT t_v.area_id  as areas_fom_vac
        ,t_a.name
FROM vacancies as t_v

join areas as t_a
on t_v.area_id=t_a.id

group by t_v.area_id,t_a.id 
order by t_a.id  asc

''' 

df_areas_in_vacancies = pd.read_sql_query(query_areas_in_vacancies, connection)
df_areas_in_vacancies

,areas_fom_vac,name
0,1,Москва
1,2,Санкт-Петербург
2,3,Екатеринбург
3,4,Новосибирск
4,7,Австрия
...,...,...
764,6363,Вольно-Надеждинское
765,6365,Актогай
766,6368,Большие Вязёмы
767,6374,Софьино (Раменский район)


**Справочник Регионы**
- Справочник никак не структуриван. Города, страны, регионы, деревни - все в перемешку без какой-бы то ни было структуры. При такой структуре данных - очень сложно осуществлять работу кадрового агентсва (из данных - невозможно понять нужно ли жителям Софьино(Раменский райнон) предлагать работу в москве, или нет). Так-же из данных такой структуры - очень сложно делать сводную аналитику(Если нужно например сделать анализ рынка по сибирскому федеральному округу, или по подмосковью - нужно в ручную перечилить все населенные пункты, которые относятся к тому или иному макрорегиону).
- Важно отметить, что справочник содержит только настоящие топонимы. Вероятно, где-то за пределами предоставленных нам данных есть механизм проверки достоверности топонима.
- Заполнен полностью без NULLов
- В справочнике 1362 наименований регионов. Только в 790 из них есть вакансии(58%)
**Рекомендация**
  
  Очень нужна структуризация справочника, или дополнительный справочник, позволяющий относить конкретное название региона к одной или более грппировкам. Условно "Москва" может входить в  следующие смысловвые группы "Россия","Европейская часть Россия", "Центральный федеральный округ", "Москва и подмосковье", "города миллионники", "СНГ".....
  
  Думаю, что что-то подобное есть у hh.ru, просто это нам не предоставляется, так как это их наработки. 

------------------


**4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).**


In [26]:
# текст запроса
query_3_4 = f'''SELECT count(distinct name) 
FROM industries
'''
# результат запроса
cnt_industries = pd.read_sql_query(query_3_4, connection).iloc[0,0]

print(f'Количество сфер деятельности, посчитанное SQL запросом: {cnt_industries}') # 294


Количество сфер деятельности, посчитанное SQL запросом: 294


In [27]:
#Выводим 5 верхних значений таблицы industries для наглядности данных 
query_industries = f''' SELECT * 
FROM industries 

'''
df_industries = pd.read_sql_query(query_industries, connection)
df_industries.head(5)


,id,name
0,7.540,Разработка программного обеспечения
1,7.539,"Системная интеграция, автоматизации технологи..."
2,27.550,Безалкогольные напитки (производство)
3,27.551,"Безалкогольные напитки (продвижение, оптовая т..."
4,13.664,Управление и эксплуатация недвижимости


In [28]:
#  Получаем краткое описание таблицы areas
df_industries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      294 non-null    object
 1   name    294 non-null    object
dtypes: object(2)
memory usage: 4.7+ KB


In [29]:
# ID Отрасли оказался дробным (для обычного идентификатора это не характерно). Предположу, что это соответсвует какой-то классификации, 
# где цифры до запятой обозначают отраслевые группы, а после запятой - подгруппы. Это не ОКВЭД и не GICS, 
#возможно какая-то внутренняя система классификации
query_industries = f''' SELECT * 
FROM industries 
order by id asc

'''

df_industries = pd.read_sql_query(query_industries, connection)
df_industries.head(25)


,id,name
0,11.452,Издательская деятельность
1,11.453,"Производство мультимедиа, контента, редакторск..."
2,11.454,"Распространение телепрограмм, кино (кабельное ..."
3,11.455,Киностудии и студии звукозаписи
4,11.456,Теле- и радиовещание
5,11.457,Продюсерский центр
6,11.458,Распространение мультимедиа и печатной продукции
7,11.459,"Маркетинговые, рекламные, BTL, дизайнерские, E..."
8,11.460,"Производство печатной, полиграфической продукции"
9,11.685,Производство и продажа рекламно-сувенирной про...


**Краткие выводы про 'industries':**

Справочник заполнен и структурирован. ID - очиведно отсылат к какому-то классификатору, но не к ОКВЭД и не к GICS

Спарвочник industries в связке с employers_industries - является хоорошим примером тог, как может быть решена(и вероятно решена в hh.ru) проблема с неструктурированностью данных в таблицах описаных выше


------------------------------

In [30]:
# создаем запрос для дополнительного сводного анализа данных
query_3_extra = f'''
SELECT  'vacancies' as name_of_the_table        
        ,sum(id_cnt) as id_cnt
        ,sum(case when more_than_one_unique_id=0 then 1 else 0 end) as all_ids_are_unique_flag
        ,sum(unique_names) unique_names
        ,sum(case when id_cnt>1 then 1 else 0 end ) as cnt_cases_when_more_then_one_id_per_name
        ,sum(number_of_NaN_in_names) number_of_NaN_in_names
        ,avg(id_cnt) as avg_ids_per_name
        ,max(id_cnt) as max_ids_per_name
        
        
FROM       (SELECT   count(id) as id_cnt
                    ,count(id)-count(distinct id) more_than_one_unique_id
                    ,count(distinct name) unique_names
                    ,count(*)-count(name) number_of_NaN_in_names
                    
                    
            FROM vacancies
            group by name) as t_vac

UNION      

SELECT  'employers' as name_of_the_table        
        ,sum(id_cnt) as id_cnt
        ,sum(case when more_than_one_unique_id=0 then 1 else 0 end) as all_ids_are_unique_flag
        ,sum(unique_names) unique_names
        ,sum(case when id_cnt>1 then 1 else 0 end ) as cnt_cases_when_more_then_one_id_per_name
        ,sum(number_of_NaN_in_names) number_of_NaN_in_names
        ,avg(id_cnt) as avg_ids_per_name
        ,max(id_cnt) as max_ids_per_name
        
        
FROM       (SELECT   count(id) as id_cnt
                    ,count(id)-count(distinct id) more_than_one_unique_id
                    ,count(distinct name) unique_names
                    ,count(*)-count(name) number_of_NaN_in_names
                    
                    
            FROM employers
            group by name) as t_emp

UNION      

SELECT  'areas' as name_of_the_table        
        ,sum(id_cnt) as id_cnt
        ,sum(case when more_than_one_unique_id=0 then 1 else 0 end) as all_ids_are_unique_flag
        ,sum(unique_names) unique_names
        ,sum(case when id_cnt>1 then 1 else 0 end ) as cnt_cases_when_more_then_one_id_per_name
        ,sum(number_of_NaN_in_names) number_of_NaN_in_names
        ,avg(id_cnt) as avg_ids_per_name
        ,max(id_cnt) as max_ids_per_name
       
        
FROM       (SELECT   count(id) as id_cnt
                    ,count(id)-count(distinct id) more_than_one_unique_id
                    ,count(distinct name) unique_names
                    ,count(*)-count(name) number_of_NaN_in_names
                    
                    
            FROM areas
            group by name) as t_areas

UNION      

SELECT  'industries' as name_of_the_table        
        ,sum(id_cnt) as id_cnt
        ,sum(case when more_than_one_unique_id=0 then 1 else 0 end) as all_ids_are_unique_flag
        ,sum(unique_names) unique_names
        ,sum(case when id_cnt>1 then 1 else 0 end ) as cnt_cases_when_more_then_one_id_per_name
        ,sum(number_of_NaN_in_names) number_of_NaN_in_names
        ,avg(id_cnt) as avg_ids_per_name
        ,max(id_cnt) as max_ids_per_name
        
        
FROM       (SELECT   count(id) as id_cnt
                    ,count(id)-count(distinct id) more_than_one_unique_id
                    ,count(distinct name) unique_names
                    ,count(*)-count(name) number_of_NaN_in_names
                    
                    
            FROM industries
            group by name) as t_ind     

order by id_cnt desc            
'''

df_extra = pd.read_sql_query(query_3_extra, connection)
df_extra



,name_of_the_table,id_cnt,all_ids_are_unique_flag,unique_names,cnt_cases_when_more_then_one_id_per_name,number_of_nan_in_names,avg_ids_per_name,max_ids_per_name
0,vacancies,49197.0,21223,21223.0,4788,0.0,2.318098,1761
1,employers,23501.0,23175,23175.0,244,0.0,1.014067,7
2,areas,1362.0,1362,1362.0,0,0.0,1.000000,1
3,industries,294.0,294,294.0,0,0.0,1.000000,1


***

# Краткие выводы по предварительному анализу данных:
Если не повторяться то информацию о структуре данных можно свести к следующему:
- Мы видим одну таблицу с данными (vacacies), три справочника (emlpoyers,areas,industries) и одну таблицу соответсвия (employers_industries она нужна, чтобы устанавливаль соответсвия между работодателями и отраслями)
- Все справочники заполнены полностью и без пропусков.
- Все справочники, кроме industries не структурированы.
- Для нормальной работы кадрового агентства не хватает информации об открытии/закрытии вакансии, классификатора ваканисй(название вакансии-суть работы), классификатора регионов(регион-уктрупненный регион), универсального идентификатора работодателей (у нас в базе 244 ситуации когда одинаковому названию работодателя соответвует несколько разных работодателей(ИНН или ОГРН запроосто решил бы эту проблему))
- В таблице vacancies - всего 49 197 вакансий. Это достаточно много, но есть ньюансы. 
- В таблице vacancies - весьма распространено дублировние идентичных вакансий в разных регионах. Примерно 20% вакансий - не являются вакансиями, а я вляются дубликатами вакансий в других регионах (Условно яндекс ищет тестировщика на удаленку и публикует объявление в каждом городе крупнее 30 тыс населения. При этом они ищут не 160 тестировщиков в разных уголках Россиии а одного.) Это может сильно смазывать картину вцелом, и по каким-то специальностям в частности.
- С высокой долей вероятности, перед нами не сама база данных hh.ru, а некий слайс базы данных. Поэтому для полноценного анализа - неплохо бы узнать на какую дату(или за какой период), и были ли применены какие-то фильтры.
- Отсуттсвие данных о динамики открытия и закрытия вакансий лишает нас возвожности проводить анализ того, с какой скоростью закрываются ваканисии, сколько претендентов проходит до мэтча и т.д. Какие вакансии подолгу не закрываются и почему. 
- Значительная доля полей в таблице vacancies не заполнена, либо заполнена плохо. 56% полей "Зарплата от" и 74% полей "Зарплата до" - не заполнены. 18% полей тербуемые навыки не заполнены...
- Среди наименований вакансий оромное количество вакансий, которые по сути одинаковая работа, но записана по разному.К примеру есть 79 вариаций "Специалист службы поддержки".
- Справочники areas и employers содержат примерно на 40% больше уникальных территорий и работодателей, чем задействовано в таблице вакансии (это допонительно косвенно подтверждает, что мы имеем дело со слайсом базы данных).
- В базе данных отсутсвует очень важная часть данных. Мы не видим количество резюме соискателей по типам должностей навыку и опыту. Без этого невозможно сделать выводы о том наш рынок труда - "рынок работника"/"нейтральный"/или рынок "работодателя". 49 000 вакансий это много, или мало? Если им соответсвует 30 000 соискателей - то много, если 1 000 000 соикателей - очень мало.
- Согласно Росстату количество работников в Росии в 2024 составило 74.8 миллионов человек. Количество безработных - 1.8 млн человек.

---------------------------------------------------------------

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [31]:
# текст запроса
query_4_1 = f'''
SELECT  t_a.name
        ,count(t_v.id) as vacancies_cnt
FROM vacancies as t_v

left join areas as t_a
on t_v.area_id=t_a.id

group by  t_a.name
order by vacancies_cnt desc

''' 
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

,name,vacancies_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [32]:
# текст запроса
query_4_2 = f'''
SELECT count(*)
FROM vacancies 
where (salary_from is not null) or (salary_to is not null)
'''
# результат запроса
cnt_4_2 = pd.read_sql_query(query_4_2, connection).iloc[0,0]
print(f'Количество вакансий, для которых заполнено хотя бы одно из двух полей с зарплатой: {cnt_4_2}, что составляет {round(cnt_4_2/cnt_vacancies*100,2)}% от общего числа вакансий.')

Количество вакансий, для которых заполнено хотя бы одно из двух полей с зарплатой: 24073, что составляет 48.93% от общего числа вакансий.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [33]:
# текст запроса
query_4_3 = f'''
SELECT round(avg(salary_from),0) as avg_salary_from
       ,round(avg(salary_to),0) as avg_salary_to
FROM vacancies 

'''
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [34]:
#  Получаем описание предложенных зарплатных вилок от/до 
#(количество не пустых, среднее, среднеквадратичное отклонение, min значение, maх значение, разбивка по квартилям) :
df_vacancies[['salary_from','salary_to']].describe()

,salary_from,salary_to
count,21485.000000,12690.000000
mean,71064.657901,110536.741923
std,56646.897132,89104.795997
min,30.000000,60.000000
25%,35000.000000,50000.000000
50%,50000.000000,80000.000000
75%,90000.000000,150000.000000
max,1000000.000000,1000000.000000


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [35]:
# текст запроса
query_4_4 = f'''
SELECT  employment as employment
        ,schedule as schedule
        ,count(id) as vacasies_cnt
FROM vacancies 
group by employment, schedule
order by vacasies_cnt desc
'''
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,employment,schedule,vacasies_cnt
0,Полная занятость,Полный день,35367
1,Полная занятость,Удаленная работа,7802
2,Полная занятость,Гибкий график,1593
3,Частичная занятость,Удаленная работа,1312
4,Полная занятость,Сменный график,940
5,Стажировка,Полный день,569
6,Полная занятость,Вахтовый метод,367
7,Частичная занятость,Полный день,347
8,Частичная занятость,Гибкий график,312
9,Проектная работа,Полный день,141


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [36]:
# текст запроса
query_4_5 = f'''
SELECT  experience as experience
        ,count(id) as vacasies_cnt
FROM vacancies 
group by experience
order by vacasies_cnt asc
'''
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,experience,vacasies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

**Выводы по детальному анализу вакансий**
- Анализ количества вакансий по регионам показывает довольно ожидаемую тенденцию: наибольшее количество вакансий сконцентрировано в крупных городах.
- Количество вакансий, для которых заполнено хотя бы одно из двух полей с зарплатой: 24073, что составляет 48.93% от общего числа вакансий. При этом  74% полей "Зарплата до" - не заполнены.
- Средние значения полей 'зарплата от' 'зарплата до' составляют 71 065 и 110 537 рублей соответсвенно. Правдв нужно учитывать, что среднеквадратичное отклонение для этих же полей составляет 56 646 и 89 104, что свидетельсвует об огромном разбросе значений как вверх так и вниз от матожидания. А медианные занчения находятся на 50 000 и 80 000 соответсвенно.
- Анализ количества вакансий в разрезе тип занятости/расписание - дал нам вполне ожидаемый результат. Мы смотрим на вакансии(пожелания работодателя). Не удивительным образом 70% работодателей хотят видеть своих работников на полной занятости/полный рабочий день, еще 16% работодателей согласны на полную занятость и удаленку, еще по три процена согланы на полную занятость при гибком графике, и не частичную занятость при удаленке. На все остальные типы занятости и графиков приходится примерно 7% вакансий. Если предположить, что наш рынок труда - рынок работодателя - то надо быть готовым ходить в офис каждый рабочий день.
- Анализ вакансий по типу требуемого опыта показал, что самое массовое предложение для соискателей с опытом от 1 до 3 лет, за ним следует 3-6 лет, очень сложно без опыта (одновременно мало вакансий и много желающих), меньше всего предложений для работников с опытом более 6 лет (это скорее всего связано с тем, что для таких вакансий поиск соискателей идет через другие каналы) 

  **Что можно было бы еще проанализировать:**
- Везде ли количество вакансий соотносится численностью населения города, или есть всплески/ провалы?
- Вызывает ли соседсво с крупным городом увеличение количества вакансий в соседних населенных пунктах (Москва-Подмосковье, СПБ - Лен область, и т.д.)?
- Для какого типа занятости/типа вакансии/должности/опыта чаще не указана нижняя или верхняя рганица зарплтатной вилки?(можно построить тепловую карту)
- Какие факторы больше всего влияют внутри однородных вакансий на размер оплаты труда (опыт/количество навыков/регион предприятия/регион вакансии)?
- Какие укрупненные вакансии приносят больше денег при сопоставимом опыте? (Условно для IT можно сравнивать Разработку/Аналитику/DS(ML)/Тестировку а разбивке по опыту строить столбчатые диаграмы) 

----------------------------------

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [37]:
# текст запроса
query_5_1 = f'''
select '1' as num_by_vacancies_cnt 
       ,*
from (
        SELECT  t_e.name
                , count(t_v.id) as vacancies_cnt
                
        FROM employers as t_e 
        
        left join vacancies as t_v
        on t_v.employer_id=t_e.id
        
        group by  t_e.name
        order by vacancies_cnt desc
        limit 1 
        ) as t_1
union all        
select '5' as num_by_vacancies_cnt 
       ,*
from (
        SELECT  t_e.name
                , count(t_v.id) as vacancies_cnt
                
        FROM employers as t_e 
        
        left join vacancies as t_v
        on t_v.employer_id=t_e.id
        
        group by  t_e.name
        order by vacancies_cnt desc
        limit 1 offset 4
        ) as t_2

''' 
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

,num_by_vacancies_cnt,name,vacancies_cnt
0,1,Яндекс,1933
1,5,Газпром нефть,331


In [38]:
# Top 10 компаний по количеству размещенных вакансий
# текст запроса
query_5_1_1 = f'''
SELECT  t_e.name
        , count(t_v.id) as vacancies_cnt
        
FROM employers as t_e 

left join vacancies as t_v
on t_v.employer_id=t_e.id

group by  t_e.name
order by vacancies_cnt desc
limit 10 

''' 
# результат запроса
df_5_1_1 = pd.read_sql_query(query_5_1_1, connection)
df_5_1_1

,name,vacancies_cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [39]:

#  Запрос, который для каждого региона выведет количество работодателей и вакансий в нём. 
# Вопрос про количество работодателей в регионе - может иметь двоякое прочтение. 
# Количество работадателей зарегистрированных в регионе (та информеция, которая хранитеся в столбце area таблицы emloyers ), 
# или количество уникальных работодателей среди вакансий в регионе 
# (некоторые работодатели являются работодателями  в 1регионе(ищут сотрудников в данном регионе), но не зарегистрированы в нем). 
# К примеру яндекс является работодателем в 181 регионе, но зсрегистрирован только в 1). 

# текст запроса вариант 1 (Количество работадателей зарегистрированных в регионах )
query_5_2_1_1 = f'''
with area_emps as(
                SELECT  t_a.name as area
                        ,count(t_e.id) as employers_cnt
                        
                FROM areas as t_a
                
                left join employers as t_e
                on t_a.id=t_e.area
                group by t_a.name
                )
     ,area_vacs as(
                 SELECT  t_a.name as area
                        ,count(t_v.id) as vacancies_cnt
                        
                FROM areas as t_a
                
                left join vacancies as t_v
                on t_a.id=t_v.area_id
                group by area
                )
Select area_emps.area
        ,employers_cnt
        ,vacancies_cnt
from area_emps as area_emps
left join area_vacs as area_vacs
on area_vacs.area=area_emps.area
order by employers_cnt	desc               
''' 

# текст запроса вариант 2 (Количество уникальных работодателей среди вакансий в регионе )
query_5_2_1_2 = f'''
with vac_area_cnts as(
                SELECT  area_id as area_id
                        ,count(distinct employer_id) as employers_cnt
                        ,count(id) as vacancies_cnt                    
                FROM vacancies
                group by area_id
                )
Select name
     ,employers_cnt
     ,vacancies_cnt
from vac_area_cnts as vac_area_cnts
left join areas as areas
on vac_area_cnts.area_id=areas.id
order by employers_cnt	desc          
''' 



# Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей
query_5_2_2 = f'''
SELECT  t_a.name as region
        ,count(t_e.id) as employers_cnt
        ,count(t_v.id) as vacancies_cnt
FROM vacancies as t_v

right join areas as t_a
on t_v.area_id=t_a.id

left join employers as t_e
on t_a.id=t_e.area

group by  region
having count(t_v.id)=0 and count(t_e.id)!=0
order by employers_cnt desc

''' 




In [40]:
# результат запроса первой части задания 5.2 вариант 1
df_5_2_1_1 = pd.read_sql_query(query_5_2_1_1, connection)
df_5_2_1_1


,area,employers_cnt,vacancies_cnt
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698
...,...,...,...
1357,Урень,0,1
1358,Уруссу,0,0
1359,Урюпинск,0,1
1360,Усмань,0,0


In [41]:
# результат запроса первой части задания 5.2 вариант 2
df_5_2_1_2 = pd.read_sql_query(query_5_2_1_2, connection)
df_5_2_1_2


,name,employers_cnt,vacancies_cnt
0,Москва,2320,5333
1,Санкт-Петербург,1451,2851
2,Минск,851,2112
3,Новосибирск,802,2006
4,Алматы,798,1892
...,...,...,...
764,Первомайское (Ленинградская область),1,1
765,Невьянск,1,1
766,Шлиссельбург,1,1
767,Краснообск,1,1


In [42]:
# результат запроса второй части задания 5.2 
df_5_2_2 = pd.read_sql_query(query_5_2_2, connection)
df_5_2_2

,region,employers_cnt,vacancies_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
181,Удобная,1,0
182,Майна (Ульяновская область),1,0
183,Верхнетуломский,1,0
184,Жаркент,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [43]:
# текст запроса
query_5_3 = f'''
SELECT  t_e.name as employer
        , count(distinct t_v.area_id) as areas_cnt
        
        
        
FROM employers as t_e

left join vacancies as t_v
on t_v.employer_id=t_e.id



group by employer
order by areas_cnt desc


''' 


In [44]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,employer,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [45]:
# текст запроса
query_5_4 = f'''
SELECT  count(t_e.name) as employer_cnt
FROM employers as t_e

left join employers_industries as t_e_i
on t_e_i.employer_id=t_e.id

--left join industries as t_i
--on t_e_i.industry_id=t_i.id

where t_e_i.industry_id is null


''' 
# результат запроса
cnt_5_4 = pd.read_sql_query(query_5_4, connection).iloc[0,0]
print(f'Количество предприятий у которых не указана сфера деятельности {cnt_5_4}')

Количество предприятий у которых не указана сфера деятельности 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [46]:
# текст запроса
query_5_5 = f'''
SELECT  t_e.name as employer
       --,count(t_e_i.industry_id)
FROM employers_industries as t_e_i

left join employers as t_e
on t_e_i.employer_id=t_e.id

group by employer
having count(t_e_i.industry_id)=4
order by employer asc
limit 1 offset 2

''' 
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,employer
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [47]:
# текст запроса
query_5_6 = f'''
SELECT  count(t_e.name) as cnt
       
FROM employers_industries as t_e_i

left join employers as t_e
on t_e_i.employer_id=t_e.id

left join industries as t_i
on t_e_i.industry_id=t_i.id


where t_i.name='Разработка программного обеспечения' --'%азработк%' and t_i.name like '%рограммного %' and t_i.name like '%беспечени%'
''' 
# результат запроса
cnt_5_6 = pd.read_sql_query(query_5_6, connection).iloc[0,0]
print(f'Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения»: {cnt_5_6}')

Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения»: 3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [48]:
# код для получения списка городов-милионников
# получаем таблицу со страницы в Википедии и передаем ее в датафрейм city_df
url='https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
data = pd.read_html(url)
city_df=data[0]
# делаем из столбца датафрейма фписок городав
city_list = list(city_df['Город'])

# заполняем строку, которую потом вставим в текст запроса  в качестве условия where.... 
where_line="where ("
for city in city_list:
    where_line=where_line+" t_a.name = '"+city+"' "
    if city!=city_list[-1]:
        where_line=where_line+"or"
    else:
        where_line=where_line+")"

# задаем по какому работодателю будет фильтрация
employer="'Яндекс'"

# текст запроса
query_5_7 = f'''
with employer_in_big_cities as( 
            SELECT  t_a.name as cities
                    ,count(t_v.id) vacancies_cnt
            FROM areas as t_a
            
            join vacancies as t_v
            on t_a.id=t_v.area_id
            
            left join employers as t_e
            on t_e.id=t_v.employer_id
            
            {where_line} and t_e.name= {employer}
            group by cities
            order by vacancies_cnt desc
            )


select   cities as name
        ,vacancies_cnt as cnt
from employer_in_big_cities
union ALL
select 'Total' as name
        ,sum(vacancies_cnt) as cnt
from employer_in_big_cities


''' 
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,name,cnt
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


***

**Выводы по анализу работодателей**
- Top 10 компаний по количесву размещенных вакансий - Яндекс	(1933),	Ростелеком (491), Тинькофф (444), СБЕР (428), Газпром нефть (331) ИК СИБИНТЕК	(327), МТС	(292), DataArt (247), Совкомбанк Технологии	(204), Первый Бит(176). Все из них, кроме Газпром-нефти - продуктовые IT компании. Предположу, что IT компаниям проще найти взаимопонимание  с IT компанией hh.ru. Плюс скорее всего IT специалиста будут искать себе работу через онлайн агрегатора вакансий, нежели чем через другие каналы.
- "регион" в справочнике работадателя "регион" в таблице вакансии - "ложные друья аналитика". Они имеют одинаковые названия, и интуитивно хочется сопоставить одни с другими и сделать какие-то выводы. На самом деле - эти регионы разные по смыслу. "регион" в справочнике работадателя - это условно место прописки предприятия, "регион" в таблице вакансии - это место, где фактически ищется работник. Яндекс, например на вю росиию один, а нанимает работников в разных регионах. Большое количество работодателей зарегистрированных в регионе не обязательно говорит о большом количестве вакансий в этом регионе, и наоборот.
-  Количество предприятий у которых не указана сфера деятельности 8419. Это треть от общего размере справочника работодателей. Самое простое объяснение состоит в том, что это поле работодателю не нужно. Оно никак не влияет на то, найдется ли соискатель на дожность или нет. Это поле нужно hh.ru для аналитики по отраслям. Незаполнненные поля - вцелом большая проблема для hh.ru.  56% полей "Зарплата от" и 74% полей "Зарплата до" - не заполнены. 18% полей тербуемые навыки не заполнены... Вероятно можно сподвигать работодателей более тчательно заполнять поля(писсимизировать объявления с незполннными полями), или самостоятельно дособирать недостающую информацию. Так информацию об основном виде деятельнсти компании можно получить в ИФНС или в Спарк. Прада нужно будет сопоставлять свой классификатор с ОКВЭД, но я думаю это не очень сложная задача.
-  Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения»: 3553. Сама по себе эта цифра мало о чем говорит. Логичнее было бы анализировать сколько вакансий IT-related. Плюс любой спрос нужно сравнивать с предложением, сколько на этих работодателей и эти вакансии приходится соискателей?
-  Яндекс много размещает вакансий в городах миллионниках... В целом - ожидаемо. Яндекс вообще много размещает вакансий. Любопытно скорее то, что количество вакансий не соответсвует линейно населению города. В москве количество вакансий всего лиш вдвое больше чем в среднем по остальным миллионникам, хотя Москва пропорционально гораздо больше.

***

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [49]:
# текст запроса
query_6_1 = f'''
SELECT  count(name) as cnt
       
FROM vacancies
where lower (name) like '%data%' or lower (name) like '%данн%'
'''

In [50]:
# результат запроса
cnt_6_1 = pd.read_sql_query(query_6_1, connection).iloc[0,0]
print(f'Количество вакансий, которые имеют отношение к данным: {cnt_6_1}')

Количество вакансий, которые имеют отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


--------------------------------------------------------------------------------
**^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^**

**!!!В описании инструкций определению вакансий для дата сайентиста закралась ошибка** 

**Помимо HTML нужно еще исключить QML и поправить тест на платформе, ато на платформе результаты не верные**

In [51]:
#!!!!! Из перечня  профессий относимых к DS стоит исключить не только HTML, но еще и QML. Разработка на QML - не дата сайенс 

query_tables_each = f''' 
SELECT *
FROM ds_vacancies
where lower(name) like '%qml%' 

'''
df_table = pd.read_sql_query(query_tables_each, connection)
df_table.head(3)

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55373225,Разработчик C++/Qt/QML,C++\tQt\tVoIP\tTCP/IP\tQML,Полный день,От 1 года до 3 лет,Полная занятость,NaN,None,76,793955
1,52995746,C++ разработчик (QML),ООП\tC++\tSQL,Полный день,От 1 года до 3 лет,Полная занятость,60000.0,None,11,115911
2,55136580,Разработчик С++/QML,Git\tQt\tLinux\tАнглийский язык\tXML,Полный день,От 1 года до 3 лет,Полная занятость,NaN,None,71,3778


In [52]:
# задаем критерии дата сайентиста
vacancies=" ((name like '%ML%' and not name like '%HTML%' and not name like '%QML%' ) or (lower (name) like '%data scientist%') or (lower (name) like '%data science%') or (lower (name) like '%исследователь данных%') or (lower (name) like '%machine learning%') or (lower (name) like '%машинн%обучен%')) "
# задаем критерии junior
junior=" (lower (name) like '%junior%' or experience = 'Нет опыта' or lower(employment) like '%cтажировка%') "

# текст запроса
query_6_2 = f'''
SELECT  count(id) 
        
       
FROM vacancies
where {junior} and {vacancies}
'''
# результат запроса
cnt_6_2= pd.read_sql_query(query_6_2, connection).iloc[0,0]
DS_tot_cnt=pd.read_sql_query(f'SELECT  count(id) FROM vacancies where {vacancies}',connection).iloc[0,0]
print(f'Всего вакансий Дата сайенс: {DS_tot_cnt}')
print(f'Есть {cnt_6_2} подходящих вакансий для начинающего дата-сайентиста, что сотсавляет {round((cnt_6_2/DS_tot_cnt)*100,2)}% от общего числа вакансий DS')

Всего вакансий Дата сайенс: 467
Есть 51 подходящих вакансий для начинающего дата-сайентиста, что сотсавляет 10.92% от общего числа вакансий DS


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [53]:
# Правильный текст запроса Мы используем ранее установленный критерий vacancies
key_skills= " (key_skills like '%SQL%'  or key_skills like '%Postgres%') "
query_6_3 = f'''
SELECT  count(id) 
       
FROM vacancies
where {key_skills} and {vacancies}
'''
# результат запроса
cnt_6_3= pd.read_sql_query(query_6_3, connection).iloc[0,0]
print(f'Для {cnt_6_3} вакансий требуется  SQL или Postgres, что сотсавляет {round((cnt_6_3/DS_tot_cnt)*100,2)}% от общего числа вакансий DS')

Для 200 вакансий требуется  SQL или Postgres, что сотсавляет 42.83% от общего числа вакансий DS


In [54]:
# Текст запроса, который даст неверный результат, но удовлетворяющий тесту на платформе SF. 
# Здесь мы для примера будем использовать не верный вариант критерия vacancies1 
#Ощибка возникает если в качестве условия использовать lower (name) like '%ml%' или name ilike '%ML%'
#В таком случае в выборку попадают 29 лишних вариаций TeamLead никак не связанных DS
vacancies1=" ((lower (name) like '%ml%' and not lower (name) like '%html%') or (lower (name) like '%data scientist%') or (lower (name) like '%data science%') or (lower (name) like '%исследователь данных%') or (lower (name) like '%machine learning%') or (lower (name) like '%машинн%обучен%')) "
key_skills= " (key_skills like '%SQL%'  or key_skills like '%Postgres%') "

query_6_3_wrong = f'''
SELECT  count(id) 
       
FROM vacancies
where {key_skills} and {vacancies1}
'''
# результат запроса
cnt_6_3_wrong= pd.read_sql_query(query_6_3_wrong, connection)
print(f'!!!Для {cnt_6_3_wrong.iloc[0,0]} вакансий требуется  SQL или Postgres.\n!!!Это не верный результат, но такого требует тест на платформе SF\n!!!Ощибка возникает если в качестве условия использовать lower (name) like %ml% или name ilike %ML%\n!!!В таком случае в выборку попадают 29 лишних вариаций TeamLead никак не связанных DS' )


!!!Для 229 вакансий требуется  SQL или Postgres.
!!!Это не верный результат, но такого требует тест на платформе SF
!!!Ощибка возникает если в качестве условия использовать lower (name) like %ml% или name ilike %ML%
!!!В таком случае в выборку попадают 29 лишних вариаций TeamLead никак не связанных DS


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [55]:

# текст запроса
key_skills= " (lower (key_skills) like '%python%') "

query_6_4 = f'''
SELECT count(id)
       
FROM vacancies
where {key_skills} and {vacancies}
'''
# результат запроса
cnt_6_4= pd.read_sql_query(query_6_4, connection).iloc[0,0]
print(f'Для {cnt_6_4} вакансий требуется  Python, что сотсавляет {round((cnt_6_4/DS_tot_cnt)*100,2)}% от общего числа вакансий DS')

Для 351 вакансий требуется  Python, что сотсавляет 75.16% от общего числа вакансий DS


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [56]:
# текст запроса
query_6_5 = f'''
SELECT  round(avg(array_length(string_to_array(key_skills,'\t'),1)),2) cnt_key_skills
       
        ,round(avg(length(key_skills )-length(REPLACE(key_skills, CHR(9), ''))),2)  cnt_key_skills_by_hint
        , count(id)
       
FROM vacancies
where  {vacancies}  --and key_skills is not null  and key_skills !=''
'''
cnt_6_5= pd.read_sql_query(query_6_5, connection)
cnt_6_5

,cnt_key_skills,cnt_key_skills_by_hint,count
0,6.43,5.43,467


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [57]:
# текст запроса
query_6_6 = f'''
SELECT  experience
        , count(id) as vacacies_cnt_total
        , sum(case  when salary_from is not null or salary_to is not null then 1 else 0 end ) as vacacies_with_salary_cnt
        , (sum(case  when salary_from is not null or salary_to is not null then 1 else 0 end )::float/count(id))*100 as vacacies_with_salary_percent
        , round(avg(case  when (salary_from is not null and salary_to is not null) then (salary_from+salary_to)/2 
                when (salary_from is not null and salary_to is  null) then salary_from
                when (salary_from is null and salary_to is not null) then salary_to
                end),0) as salary
           
FROM vacancies
where  {vacancies}  
group by experience
'''
# результат запроса
cnt_6_6= pd.read_sql_query(query_6_6, connection)
cnt_6_6

,experience,vacacies_cnt_total,vacacies_with_salary_cnt,vacacies_with_salary_percent,salary
0,Более 6 лет,34,0,0.000000,NaN
1,Нет опыта,28,7,25.000000,74643.0
2,От 1 года до 3 лет,219,27,12.328767,142626.0
3,От 3 до 6 лет,186,27,14.516129,243115.0


***

**Выводы по предметному анализу:**
- Общее количество вакансий на hh.ru, которые имеют отношение к данным - 1771. Сама по себе цифра висит в воздухе и ничего не говорит. Много это или мало можно тослько сказать сранению счем-то. Например, если бы была динамика, то можно было-бы говорить о том растет или падает спрос на специалистов по данным. В общем количестве вакансий - это 3.6%. Программистов, например, 7412 вакансий, тех поддержки - 2423	, сис админов -2000. Важно еще понимать, что из 1771 только 467 вакансий дата сайентистов всех уровней, остальное это аналитики и специалисты по организации хранения данных.
- Есть 51 подходящих вакансий для начинающего дата-сайентиста, что сотсавляет 10.92% от общего числа вакансий дата сайентистов. Что тут можно сказать, тяжело быть начинающим датасайентистом. Куда проще быть начинающим программистом, или на худой конец аналитиком.
- Для дата сайентиста - SQL является вжныйм навыком он требуется в  42.83% вакансий.
- Еще более важным навыком для дата сайентиста является python. Он требуется в 75.16% вакансий.
- Знание Python и SQL важные требования, но не достаточные. В среднем от датасайетиста требуется 6.43 ключевых навыков. (От разработчика C++ обычно требуют снания C++ :)
- Все это сочетается с не то, чтобы особо звездной зарплатой. В тех 7 вакансиях, для джунов в которых есть зарплатная вилка, средняяя зарплата получается 74 643(примерно, как в КБ в Москве). С опытом идет увеличение, но рынок  достаточно узкий и достаточно темный(зарплатная вилка от и до есть только у 13% вакансий датасайенс, что существенно меньше, чем в среднем по HH.ru)
- Исходя из анализа вакансий на HH.ru можно сделать вывод, что учиться на датасайенсе есть смысл только из любви к математике и к магии "черного ящика".
**Что можно дополнительно исследовать:**
  * Широту рынка смежных специальностей
  * Какое направление требует сколько навыков, и как это коррелирует с предлагаемыми зарплатами.
  * Какое из смежных направлений наиболее гуманно для junior-ов. Как видно из анализа вакансий - самый сложный отрезок - это от 0 до года. 
      	 


***

## Доп анализ
**Наблюдается не высокая доля указаныех зарплаит. проверяем репрезентативность по регионам и укрупненным вакансиям**

In [58]:
# текст запроса просто по региону
vacancies=" ((name like '%ML%' and not name like '%HTML%') or (lower (name) like '%data scientist%') or (lower (name) like '%data science%') or (lower (name) like '%исследователь данных%') or (lower (name) like '%machine learning%') or (lower (name) like '%машинн%обучен%')) "

query_6_7 = f'''
SELECT  t_a.name as area
        , count(t_v.id) as vacacies_cnt_total
        , sum(case  when salary_from is not null or salary_to is not null then 1 else 0 end ) as vacacies_with_salary_cnt
        , (sum(case  when salary_from is not null or salary_to is not null then 1 else 0 end )::float/count(t_v.id))*100 as vacacies_with_salary_percent
        , round(avg(case  when (salary_from is not null and salary_to is not null) then (salary_from+salary_to)/2 
                when (salary_from is not null and salary_to is  null) then salary_from
                when (salary_from is null and salary_to is not null) then salary_to
                end),0) as salary
           
FROM vacancies as t_v
left join areas as t_a 
on t_v.area_id=t_a.id
group by  t_a.name
order by vacacies_cnt_total desc
'''
# результат запроса
cnt_6_7= pd.read_sql_query(query_6_7, connection)
cnt_6_7.head(60)

,area,vacacies_cnt_total,vacacies_with_salary_cnt,vacacies_with_salary_percent,salary
0,Москва,5333,1592,29.851866,134140.0
1,Санкт-Петербург,2851,1203,42.195721,104410.0
2,Минск,2112,506,23.958333,90426.0
3,Новосибирск,2006,929,46.311067,107777.0
4,Алматы,1892,643,33.985201,73171.0
5,Екатеринбург,1698,901,53.062426,91896.0
6,Нижний Новгород,1670,680,40.718563,96059.0
7,Казань,1415,628,44.381625,92207.0
8,Краснодар,1301,693,53.266718,92328.0
9,Самара,1144,505,44.143357,80223.0


In [59]:
# Укрупняем вакансии

# {cnt_vacancies} берем ранее рассчитаное в задании количество ванансий


data_scientist=" ((name like '%ML%' and not name like '%HTML%') or (lower (name) like '%data scientist%') or (lower (name) like '%data science%') or (lower (name) like '%исследователь данных%') or (lower (name) like '%machine learning%') or (lower (name) like '%машинн%обучен%')) "
coder = "  (lower (name) like '%программист%') or (lower (name) like '%разработчик%')or (lower (name) like '%developer%')or (lower (name) like '%programist%')or (lower (name) like '%programer%')"
support ="(name like '%оддержк%')  and ((lower(name) like '%служб%')or(lower(name) like '%техн%'))"# and (name like '%пециалис%')"
sys_admin= "  ((lower (name) like '%сис%') and (lower (name) like '%админ%')) or ((lower (name) like '%sys%') and (lower (name) like '%admin%'))"
analyst= "  (lower (name) like '%анали%') or (lower (name) like '%anali%')or (lower (name) like '%analy%')"
project_men= "  (((lower (name) like '%менедж%')or (lower (name) like '%руковод%')or (lower (name) like '%администрат%') )and (lower (name) like '%проект%')) or ((lower (name) like '%project%') and (lower (name) like '%manag%'))"
product_men= "  ((lower (name) like '%менедж%') and ((lower (name) like '%продукт%')or (lower (name) like '%продакт%'))) or ((lower (name) like '%product%') and (lower (name) like '%manag%'))"
tester ="  (lower (name) like '%тестир%') or (lower (name) like '%test%')"
dev_ops="  (name like '%Dev%') and (name like '%Ops%')"
info_sec= "  ((name like '%Info%') and (name like '%Sec%')) or ((lower (name) like '%information%') and (lower (name) like '%secur%')) or ((lower (name) like '%информац%') and ((lower (name) like '%защит%')or (lower (name) like '%безоп%')or (lower (name) like '%охран%')))"
query_vacancies_grouped = f'''
SELECT  case when {data_scientist} then 'data_scientist' 
             when {coder} then 'Программист' 
             when {support} then 'Тех поддержка' 
             when {sys_admin} then 'Системный администратор' 
             when {project_men} then 'Project manager' 
             when {product_men} then 'Product manager' 
             when {analyst} then 'Аналитик' 
             when {dev_ops} then 'DevOps'
             when {tester} then 'Тестировщик'
             when {info_sec} then 'InfoSec'
                else name end as name_grouped
        , count(id) as vacacies_cnt_total
        , (count(id)::float)/{cnt_vacancies}*100 as vacancies_percent
        , sum(case  when salary_from is not null or salary_to is not null then 1 else 0 end ) as vacacies_with_salary_cnt
        , (sum(case  when salary_from is not null or salary_to is not null then 1 else 0 end )::float/count(id))*100 as vacacies_with_salary_percent
        , round(avg(case  when (salary_from is not null and salary_to is not null) then (salary_from+salary_to)/2 
                when (salary_from is not null and salary_to is  null) then salary_from
                when (salary_from is null and salary_to is not null) then salary_to
                end),0) as salary
           
FROM vacancies
group by  name_grouped
order by vacacies_cnt_total desc
'''
df_vacancies_grouped= pd.read_sql_query(query_vacancies_grouped, connection)
df_vacancies_grouped.head(60)

,name_grouped,vacacies_cnt_total,vacancies_percent,vacacies_with_salary_cnt,vacacies_with_salary_percent,salary
0,Программист,15654,31.819013,7412,47.348920,118374.0
1,Аналитик,6068,12.334085,1935,31.888596,90241.0
2,Тех поддержка,3080,6.260544,2423,78.668831,37976.0
3,Системный администратор,3008,6.114194,2007,66.722074,54664.0
4,Project manager,2426,4.931195,1197,49.340478,86602.0
5,Тестировщик,1198,2.435108,300,25.041736,82361.0
6,InfoSec,747,1.518385,433,57.965194,59466.0
7,DevOps,595,1.209423,181,30.420168,211120.0
8,Product manager,533,1.083399,188,35.272045,111416.0
9,data_scientist,480,0.975669,62,12.916667,177379.0


## Общий вывод по проекту

Проект исследования носил учебный характер, на примере слайса данных HH.ru мы провели анализ структуры данных, а так-же самих данных, касающихся  различных аспектов рынка труда.
Мы отработали навыки групировки данных, вычлнения данных, сопоставления одних данных с другими.
Мы узнали какого типа вакансий больше всего на рынке. Выяснили среднии зарплаты для различных вакансий и различных сочетаний опыта и навыков.
Выяснили, что в москве гораздо чаще, чем в среднем по России работодатели не указывают зарплатную вилку. Выяснили, что для датасайентистов почти самый низкий по рынку процент вакансий с указанной зарплатной вилкой.
Сформулировали рекомендации по улучшению анализируемой базы данных. Выдвинули гипотезу(и получили косвенные подтверждения), что перед нами не база данных HH.ru а все-лишь слайс базы данных с усеченной структурой данных. 

**Что можно еще исследовать**

Основные направления дополнительных исследований связаны с преодолением ассиметрии информации:

- Мы не видим исходной структуры данных, по этому не можем проанализировать, насколько она оптимальна для хранения данных и получения аналитики. При наличи доступа к исходной структуре данных - можно разработать рекомендации по улучшению аналитики, а так-же рекомендации по увеличению количества заполненных полей...
- Мы не знаем к какому периоду относится слайс данных, по этому сложно делать выводы актуальном состоянии рынка труда
- Мы не знаем какие применялись фильтры к данным, поэтому сложно делать выводы о доле HH.ru на рынке рекрутинга в России. При наличии более полных данных можно было-бы проанализировать занимаемые нишы и разработать рекомендации по расширению.
- В дазе данных отсутсвует информация о соискателях, соответсвенно мы видим рынок только глазами работодателя (что он хочет, и сколько готов платить при наличии определенных навыков). При этом на рынке присутсвуют две группы сторон работодатели и соискатели. При наличии информации о соискателях - можно рассчитывать, сколько соискателей какого уровня приходится на какую группу вакансий. Смотреть не только какие навыки требуют работодатели, но и какими навыками обладают претенденты. Из анализа совокупности этих данных, можно вырабатывать рекомендации претендентам и работодателям.
- Мы не видим никакую динамику. При наличии информации в динамике - можно анализировать скорость пояления и закрытия вакансий, уровень удовлетворенности работников и работодателей. и т.д. 







In [60]:
# не забываем закрыть соединение после окончания работы
connection.close()